# Este projeto tem como objetivo responder algumas curiosidades a respeito dos dados de bilheteria diária divulgados mensalmente pela ANCINE.

## Nesta primeira parte, nos esforçaremos para:
1.   Importar os dados
2.   Organizá-los de maneira mais favrável para a análise
3.   Realizar a exportação tabela, deixando-a com um tamanho mais adequado para a análise.



In [ ]:
# Instalando e cinfigurandos pacotes a serem utilizados durante o projeto:
import pandas as pd
pd.set_option('display.max_columns', None)
from zipfile import ZipFile
import zipfile
import numpy as np
from io import StringIO
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt

# Configurando o Google Drive para uso no Collab:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Organiando dados na área de trabalho:
with zipfile.ZipFile('/content/drive/MyDrive/br_ancine_diarias_de_cinema/input/bilheteria-diaria-obras-por-distribuidoras-csv.zip') as zip:
    zip.extractall('bilheteria')

In [ ]:
# Criando uma função de importação de arquivos:
def import_etl(path:str):
  name = path.split("/")[-1].split(".")[0]
  df = pd.read_csv(path, dtype=str, sep =";", na_values="")
  df['DATA_EXIBICAO'] = df['DATA_EXIBICAO'].str.replace('/', '-')
  df['DATA_EXIBICAO']  = pd.to_datetime(df['DATA_EXIBICAO'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')
  df['CNPJ_DISTRIBUIDORA'] = df['CNPJ_DISTRIBUIDORA'].str.replace('.', '', regex=True).replace('/', '', regex=True).replace('-', '', regex=True)

  return df

In [ ]:
# Indicando o caminho dos arquivos:
path = "/content/bilheteria/"
files = os.listdir(path)
files.sort()

In [ ]:
# Criando uma iteração para auxiliar na concatenação dos arquivos:
dfs = []
for file in files:
    dfs.append(import_etl(path+file))

In [ ]:
#Concatenando, reorganizando colunas e exportando arquivo com histórico geral das bilheterias:
df_geral = pd.concat(dfs, axis=0)
df_geral.columns= df_geral.columns.str.lower()
df_geral = df_geral.drop(columns=["registro_complexo", "registro_grupo_exibidor", "registro_exibidor", "cnpj_distribuidora", "cpb_roe", "titulo_brasil", "registro_sala", "municipio_sala_complexo"])
ordem = ['data_exibicao',
          'uf_sala_complexo',
          'pais_obra',
          'titulo_original',
          'publico']
df_geral = df_geral[ordem]
df_geral = df_geral.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_geral['ano_exibicao'] = df_geral['data_exibicao'].str[:4]
df_geral['dia_exibicao'] = df_geral['data_exibicao'].str[-2:]
df_geral['mes_exibicao'] = df_geral['data_exibicao'].str[5:7]
df_geral.to_parquet('/content/drive/MyDrive/br_ancine_diarias_de_cinema/output/bilheteria_geral', index=False)